<a href="https://colab.research.google.com/github/Biscuitkru/CS5228_Project/blob/main/CS5228_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import os, re, math, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from typing import Tuple, Dict, List
from collections import Counter

In [139]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [140]:
########################################
# Configs
########################################

# Main
train_path = "/content/drive/MyDrive/CS Masters/CS5228/train.csv"
test_path = "/content/drive/MyDrive/CS Masters/CS5228/test.csv"

# Auxiliary data
hdb_data_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-hdb-block-details.csv"
gov_hawkers_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-gov-hawkers.csv"
mrt_stations_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-mrt-stations.csv"
pri_schools_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-primary-schools.csv"
sec_schools_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-secondary-schools.csv"
shopping_malls_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-shopping-malls.csv"

# Modeling controls
seed = 777
np.random.seed(seed)

## Data Preprocessing

In [141]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [142]:
train_df.head()

,MONTH,TOWN,FLAT_TYPE,BLOCK,STREET,FLOOR_RANGE,FLOOR_AREA_SQM,FLAT_MODEL,ECO_CATEGORY,LEASE_COMMENCE_DATA,RESALE_PRICE
0,2020-10,woodlands,4 room,681B,woodlands drive 62,07 to 09,102.0,premium apartment,uncategorized,2000,420000.0
1,2021-07,bishan,4 room,264,bishan street 24,07 to 09,104.0,model a,uncategorized,1992,585000.0
2,2021-05,bukit panjang,4 room,520,jelapang road,19 to 21,102.0,model a,uncategorized,1998,450000.0
3,2021-08,punggol,4 room,121B,edgedale plains,16 to 18,93.0,model a,uncategorized,2017,465000.0
4,2023-05,hougang,5 room,997B,Buangkok Crescent,10 to 12,113.0,improved,uncategorized,2018,710000.0


In [143]:
train_df.columns = train_df.columns.str.lower()
test_df.columns = test_df.columns.str.lower()

In [144]:
train_df.describe() # No NA values at all

,floor_area_sqm,lease_commence_data,resale_price
count,162691.000000,162691.000000,1.626910e+05
mean,96.891250,1996.358993,5.188430e+05
std,24.030547,14.245920,1.832442e+05
min,31.000000,1966.000000,1.500000e+05
25%,82.000000,1985.000000,3.820000e+05
50%,93.000000,1996.000000,4.880000e+05
75%,112.000000,2011.000000,6.220000e+05
max,366.700000,2022.000000,1.658888e+06


In [145]:
# Date Manipulation
def process_month_column(df):
    df['month'] = pd.to_datetime(df['month'], format="%Y-%m")
    df['sale_year'] = df['month'].dt.year
    df['sale_month'] = df['month'].dt.month
    # sale_quarter as a feature to capture potential seasonality in resale prices.
    df['sale_quarter'] = df['month'].dt.quarter
    return df

train_df = process_month_column(train_df)
test_df = process_month_column(test_df)

# additional month_index to capture trend
abs_month_train = train_df["sale_year"]*12 + train_df["sale_month"]
abs_month_test  = test_df["sale_year"]*12 + test_df["sale_month"]
train_df["month_index"] = (abs_month_train - abs_month_train.min()).astype(float)
test_df["month_index"]  = (abs_month_test  - abs_month_train.min()).astype(float)

In [146]:
# To remove trailing whitespace and reduce multiple spaces to a single space
def norm_text_basic(s):
    if not isinstance(s, str):
        return s
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s

# Normalise + uppercase, mainly for the str values
def norm_key(s, case="upper"):
    s = norm_text_basic(s)
    if isinstance(s, str):
        return s.upper() if case == "upper" else s.casefold()
    return s

In [147]:
for col in ["town","flat_type", "block", "street", "flat_model"]:
    if col in train_df.columns: train_df[col] = train_df[col].map(norm_key)
    if col in test_df.columns: test_df[col]  = test_df[col].map(norm_key)

In [148]:
train_df["flat_type"].unique()

array(['4 ROOM', '5 ROOM', '3 ROOM', '5-ROOM', '3-ROOM', 'EXECUTIVE',
       '4-ROOM', '2-ROOM', '2 ROOM', '1 ROOM', 'MULTI GENERATION',
       '1-ROOM'], dtype=object)

In [149]:
test_df["flat_type"].unique()

array(['3 ROOM', '4 ROOM', '5-ROOM', '5 ROOM', '4-ROOM', 'EXECUTIVE',
       '3-ROOM', '2 ROOM', '2-ROOM', '1-ROOM', 'MULTI GENERATION',
       '1 ROOM'], dtype=object)

In [150]:
# Standardize flat type
def standardise_flat_type(val: str) -> str:
    v = val.strip().replace("_", " ").replace("-", " ").replace("  ", " ")
    v = v.replace("ROOM", " ROOM")
    v = re.sub(r"\s+", " ", v).strip()
    return v

In [151]:
train_df['flat_type'] = train_df['flat_type'].apply(standardise_flat_type)
test_df['flat_type'] = test_df['flat_type'].apply(standardise_flat_type)
print(train_df['flat_type'].unique())

['4 ROOM' '5 ROOM' '3 ROOM' 'EXECUTIVE' '2 ROOM' '1 ROOM'
 'MULTI GENERATION']


In [152]:
# General Price hierarchy for HDB flats
flat_type_order = ["1 ROOM","2 ROOM","3 ROOM","4 ROOM","5 ROOM","MULTI GENERATION","EXECUTIVE"]
order_map = {ft:i for i, ft in enumerate(flat_type_order)}

train_df["flat_type_price_hier"] = train_df["flat_type"].map(lambda x: order_map.get(str(x).upper(), np.nan))
test_df["flat_type_price_hier"]  = test_df["flat_type"].map(lambda x: order_map.get(str(x).upper(), np.nan))

# For Decision trees
for col, val in [("is_exec","EXECUTIVE"), ("is_multigen","MULTI GENERATION")]:
    train_df[col] = (train_df["flat_type"].str.upper() == val).astype(int)
    test_df[col]  = (test_df["flat_type"].str.upper() == val).astype(int)

In [153]:
# No. of Bedrooms
BEDROOMS_MAP = {"1 ROOM":0,"2 ROOM":1,"3 ROOM":2,"4 ROOM":3,"5 ROOM":3,"EXECUTIVE":3,"MULTI GENERATION":4}
train_df["bedrooms_est"] = train_df["flat_type"].str.upper().map(BEDROOMS_MAP)
test_df["bedrooms_est"]  = test_df["flat_type"].str.upper().map(BEDROOMS_MAP)

In [154]:
# Floor Range, just going to set as the middle numnber
def parse_floor_mid(floor_range) -> float:
    if isinstance(floor_range, (float)):
        return float(floor_range)
    m = re.findall(r"\d{1,2}", floor_range)
    if len(m) >= 2:
        lo, hi = int(m[0]), int(m[1])
        if lo <= hi:
            return (lo + hi) / 2.0
    return np.nan

In [155]:
train_df['floor_range'] = train_df['floor_range'].apply(parse_floor_mid).astype(float)
train_df = train_df.rename(columns={"floor_range": "floor_mid"})

test_df['floor_range'] = test_df['floor_range'].apply(parse_floor_mid).astype(float)
test_df = test_df.rename(columns={"floor_range": "floor_mid"})

print(np.sort(train_df['floor_mid'].unique()))
print(np.sort(test_df['floor_mid'].unique()))

[ 2.  5.  8. 11. 14. 17. 20. 23. 26. 29. 32. 35. 38. 41. 44. 47. 50.]
[ 2.  5.  8. 11. 14. 17. 20. 23. 26. 29. 32. 35. 38. 41. 44. 47. 50.]


In [156]:
train_df['flat_model'].unique()

array(['PREMIUM APARTMENT', 'MODEL A', 'IMPROVED', 'NEW GENERATION',
       'STANDARD', 'MAISONETTE', 'DBSS', 'SIMPLIFIED', 'APARTMENT',
       'MODEL A2', 'TYPE S1', 'ADJOINED FLAT', 'MODEL A MAISONETTE',
       'TYPE S2', '3GEN', '2 ROOM', 'TERRACE', 'PREMIUM APARTMENT LOFT',
       'PREMIUM MAISONETTE', 'MULTI GENERATION', 'IMPROVED MAISONETTE'],
      dtype=object)

In [157]:
test_df['flat_model'].unique()

array(['MODEL A', 'IMPROVED', 'SIMPLIFIED', 'APARTMENT',
       'PREMIUM APARTMENT', 'NEW GENERATION', 'STANDARD', 'MAISONETTE',
       'DBSS', 'MODEL A MAISONETTE', 'MODEL A2', 'IMPROVED MAISONETTE',
       'TYPE S1', '2 ROOM', 'ADJOINED FLAT', '3GEN', 'TYPE S2',
       'PREMIUM APARTMENT LOFT', 'TERRACE', 'PREMIUM MAISONETTE',
       'MULTI GENERATION'], dtype=object)

In [158]:
print(train_df['eco_category'].unique())
print(test_df['eco_category'].unique())

# Current dataset is taken directly from the Kaggle scoreboard which is missing eco_category (Might need to double check whether this is true)
train_df = train_df.drop('eco_category', axis=1)
test_df = test_df.drop('eco_category', axis=1)

['uncategorized']
['uncategorized']


In [159]:
# Flat age, years left from 99
train_df = train_df.rename(columns={"lease_commence_data": "lease_commence_date"})
test_df = test_df.rename(columns={"lease_commence_data": "lease_commence_date"})

train_df["lease_left"] = 99 - (train_df["sale_year"] - train_df["lease_commence_date"])
test_df["lease_left"] = 99 - (test_df["sale_year"] - test_df["lease_commence_date"])

In [160]:
train_df.head()

,month,town,flat_type,block,street,floor_mid,floor_area_sqm,flat_model,lease_commence_date,resale_price,sale_year,sale_month,sale_quarter,month_index,flat_type_price_hier,is_exec,is_multigen,bedrooms_est,lease_left
0,2020-10-01,WOODLANDS,4 ROOM,681B,WOODLANDS DRIVE 62,8.0,102.0,PREMIUM APARTMENT,2000,420000.0,2020,10,4,45.0,3,0,0,3,79
1,2021-07-01,BISHAN,4 ROOM,264,BISHAN STREET 24,8.0,104.0,MODEL A,1992,585000.0,2021,7,3,54.0,3,0,0,3,70
2,2021-05-01,BUKIT PANJANG,4 ROOM,520,JELAPANG ROAD,20.0,102.0,MODEL A,1998,450000.0,2021,5,2,52.0,3,0,0,3,76
3,2021-08-01,PUNGGOL,4 ROOM,121B,EDGEDALE PLAINS,17.0,93.0,MODEL A,2017,465000.0,2021,8,3,55.0,3,0,0,3,95
4,2023-05-01,HOUGANG,5 ROOM,997B,BUANGKOK CRESCENT,11.0,113.0,IMPROVED,2018,710000.0,2023,5,2,76.0,4,0,0,3,94


In [161]:
# Loading Aux Data
aux_dict = {
    "hdb": pd.read_csv(hdb_data_path),
    "mrt": pd.read_csv(mrt_stations_path),
    "pri": pd.read_csv(pri_schools_path),
    "sec": pd.read_csv(sec_schools_path),
    "malls": pd.read_csv(shopping_malls_path),
    "hawkers": pd.read_csv(gov_hawkers_path)
}

In [162]:
# HDB Data
hdb = aux_dict["hdb"].copy()

hdb.columns = hdb.columns.str.lower()

for col in ["town","block", "address", "subzone", "planning_area", "region"]:
    if col in hdb.columns: hdb[col] = hdb[col].map(norm_key)

In [163]:
hdb.head()

,town,block,address,postal_code,latitude,longitude,max_floor,subzone,planning_area,region
0,WOODLANDS,205,MARSILING DRIVE,730205,1.444946,103.773484,14,NORTH COAST,WOODLANDS,NORTH REGION
1,CLEMENTI,506,WEST COAST DRIVE,120506,1.312517,103.760867,12,CLEMENTI CENTRAL,CLEMENTI,WEST REGION
2,SENGKANG,267B,COMPASSVALE LINK,542267,1.383200,103.897433,17,COMPASSVALE,SENGKANG,NORTH-EAST REGION
3,WOODLANDS,521,WOODLANDS DRIVE 14,730521,1.433716,103.793326,12,WOODLANDS SOUTH,WOODLANDS,NORTH REGION
4,PASIR RIS,531,PASIR RIS DRIVE 1,510531,1.371362,103.950398,8,PASIR RIS CENTRAL,PASIR RIS,EAST REGION


In [164]:
hdb_pairs = hdb[["town","block"]].drop_duplicates()
train_pairs = train_df[["town","block"]]
test_pairs  = test_df[["town","block"]]

In [165]:
train_cov = train_pairs.merge(hdb_pairs, on=["town","block"], how="left", indicator=True)
test_cov = test_pairs.merge(hdb_pairs,  on=["town","block"], how="left", indicator=True)

train_match_n = (train_cov["_merge"] == "both").sum()
test_match_n = (test_cov["_merge"] == "both").sum()

print(f"[COVERAGE] Train matches: {train_match_n}/{len(train_cov)} = {train_match_n/len(train_cov)}")
print(f"[COVERAGE] TEST matches: {test_match_n}/{len(test_cov)} = {test_match_n/len(test_cov)}")

[COVERAGE] Train matches: 162691/162691 = 1.0
[COVERAGE] TEST matches: 50000/50000 = 1.0


In [166]:
hdb_unique = hdb.drop_duplicates(subset=["town","block"])

cols_to_add = ["latitude","longitude","max_floor","subzone","planning_area","region"]
cols_to_add = [c for c in cols_to_add if c in hdb_unique.columns]  # guard

train_df = train_df.merge(hdb_unique[["town","block", *cols_to_add]], on=["town","block"], how="left")
test_df = test_df.merge (hdb_unique[["town","block", *cols_to_add]], on=["town","block"], how="left")

In [167]:
# MRT Data
mrt = aux_dict["mrt"].copy()

mrt.columns = mrt.columns.str.lower()

for col in ["code","name", "status", "subzone", "planning_area", "region"]:
    if col in mrt.columns: mrt[col] = mrt[col].map(norm_key)

In [168]:
mrt

,code,name,status,latitude,longitude,subzone,planning_area,region
0,NS10,ADMIRALTY MRT STATION,OPEN,1.440689,103.800933,WOODLANDS EAST,WOODLANDS,NORTH REGION
1,EW9,ALJUNIED MRT STATION,OPEN,1.316442,103.882981,ALJUNIED,GEYLANG,CENTRAL REGION
2,NS16,ANG MO KIO MRT STATION,OPEN,1.370017,103.849450,CHENG SAN,ANG MO KIO,NORTH-EAST REGION
3,CR11,ANG MO KIO MRT STATION,PLANNED,1.370017,103.849450,CHENG SAN,ANG MO KIO,NORTH-EAST REGION
4,CR2,AVIATION PARK MRT STATION,PLANNED,1.370000,104.003056,CHANGI AIRPORT,CHANGI,EAST REGION
...,...,...,...,...,...,...,...,...
238,NE11,WOODLEIGH MRT STATION,OPEN,1.339167,103.870833,BIDADARI,TOA PAYOH,CENTRAL REGION
239,DT36,XILIN MRT STATION,PLANNED,1.328889,103.965000,XILIN,TAMPINES,EAST REGION
240,NS5,YEW TEE MRT STATION,OPEN,1.396986,103.747239,YEW TEE,CHOA CHU KANG,WEST REGION
241,NS15,YIO CHU KANG MRT STATION,OPEN,1.381906,103.844817,YIO CHU KANG,ANG MO KIO,NORTH-EAST REGION


In [169]:
print(mrt['status'].unique())
# Open and Planned MRT proximity features

from sklearn.neighbors import BallTree

mrt_open = mrt[mrt["status"] == "OPEN"].copy()
mrt_planned = mrt[mrt["status"] == "PLANNED"].copy()

R = 6371000.0
def to_rad(df): return np.deg2rad(df[["latitude", "longitude"]].to_numpy(dtype=float))

tree_open = BallTree(to_rad(mrt_open), metric="haversine")
tree_planned = BallTree(to_rad(mrt_planned), metric="haversine")

['OPEN' 'PLANNED']


In [170]:
def add_mrt_features(df, radii_open=(400,800,1000), radii_plan=(400,800,1000)):
    idx = df[["latitude","longitude"]].dropna().index
    X   = np.deg2rad(df.loc[idx, ["latitude","longitude"]].to_numpy(dtype=float))

    # OPEN: nearest + counts
    if tree_open is not None and len(idx):
        dist, _ = tree_open.query(X, k=1)
        df.loc[idx, "mrt_open_nearest_m"] = dist[:,0] * R
        for r in radii_open:
            df.loc[idx, f"mrt_open_within_{r}m"] = tree_open.query_radius(X, r/R, count_only=True)
    else:
        df["mrt_open_nearest_m"] = np.nan
        for r in radii_open: df[f"mrt_open_within_{r}m"] = 0

    # PLANNED: nearest + counts
    if tree_planned is not None and len(idx):
        dist, _ = tree_planned.query(X, k=1)
        df.loc[idx, "mrt_plan_nearest_m"] = dist[:,0] * R
        for r in radii_plan:
            df.loc[idx, f"mrt_plan_within_{r}m"] = tree_planned.query_radius(X, r/R, count_only=True)
    else:
        df["mrt_plan_nearest_m"] = np.nan
        for r in radii_plan: df[f"mrt_plan_within_{r}m"] = 0

    # Comparative signals
    df["mrt_any_nearest_m"] = np.nanmin(
        np.vstack([df["mrt_open_nearest_m"].to_numpy(dtype=float),
                   df["mrt_plan_nearest_m"].to_numpy(dtype=float)]),
        axis=0
    )
    df["mrt_plan_closer_than_open"] = (
        (df["mrt_plan_nearest_m"] < df["mrt_open_nearest_m"]).astype("Int64")
    )

    # fill NA counts for rows without coords
    for c in [col for col in df.columns if col.startswith(("mrt_open_within_","mrt_plan_within_"))]:
        df[c] = df[c].fillna(0).astype(int)

    return df

train_df = add_mrt_features(train_df)
test_df = add_mrt_features(test_df)

In [171]:
train_df

,month,town,flat_type,block,street,floor_mid,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,mrt_open_nearest_m,mrt_open_within_400m,mrt_open_within_800m,mrt_open_within_1000m,mrt_plan_nearest_m,mrt_plan_within_400m,mrt_plan_within_800m,mrt_plan_within_1000m,mrt_any_nearest_m,mrt_plan_closer_than_open
0,2020-10-01,WOODLANDS,4 ROOM,681B,WOODLANDS DRIVE 62,8.0,102.0,PREMIUM APARTMENT,2000,420000.0,...,306.013744,1,1,1,2269.847356,0,0,0,306.013744,0
1,2021-07-01,BISHAN,4 ROOM,264,BISHAN STREET 24,8.0,104.0,MODEL A,1992,585000.0,...,1000.891332,0,0,0,786.297948,0,1,1,786.297948,1
2,2021-05-01,BUKIT PANJANG,4 ROOM,520,JELAPANG ROAD,20.0,102.0,MODEL A,1998,450000.0,...,1052.540106,0,0,0,2144.006007,0,0,0,1052.540106,0
3,2021-08-01,PUNGGOL,4 ROOM,121B,EDGEDALE PLAINS,17.0,93.0,MODEL A,2017,465000.0,...,872.296018,0,0,1,872.296018,0,0,1,872.296018,0
4,2023-05-01,HOUGANG,5 ROOM,997B,BUANGKOK CRESCENT,11.0,113.0,IMPROVED,2018,710000.0,...,1342.862156,0,0,0,1912.846578,0,0,0,1342.862156,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162686,2017-07-01,HOUGANG,4 ROOM,708,HOUGANG AVENUE 2,2.0,91.0,NEW GENERATION,1985,335000.0,...,627.321503,0,1,2,627.321503,0,1,1,627.321503,0
162687,2020-09-01,PASIR RIS,4 ROOM,634,PASIR RIS DRIVE 1,5.0,104.0,MODEL A,1995,388000.0,...,1125.824511,0,0,0,759.006005,0,1,2,759.006005,1
162688,2017-10-01,GEYLANG,4 ROOM,319,UBI AVENUE 1,11.0,84.0,SIMPLIFIED,1985,373000.0,...,443.281824,0,1,3,3811.017617,0,0,0,443.281824,0
162689,2020-08-01,SENGKANG,5 ROOM,290B,COMPASSVALE CRESCENT,2.0,110.0,IMPROVED,2001,420000.0,...,637.643152,0,2,2,2214.204658,0,0,0,637.643152,0
